In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L1\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['MIDATL.csv', 'SOUTH.csv', 'WEST.csv']


In [ ]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[1]))

,Net,Temperature
0,12571.309,0.5
1,12449.182,0.1
2,12295.476,-0.5
3,12354.318,-0.7
4,12542.606,-0.7
...,...,...
43819,11682.100,9.9
43820,11442.870,10.2
43821,11166.770,9.6
43822,10838.550,9.0


### PROCESSING THE DATA

In [ ]:
# Restricting the load data to a single zone
fileName  = fileList[1]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_SOUTH,Temp_SOUTH
0,12571.309,0.5
1,12449.182,0.1
2,12295.476,-0.5
3,12354.318,-0.7
4,12542.606,-0.7
...,...,...
43819,11682.100,9.9
43820,11442.870,10.2
43821,11166.770,9.6
43822,10838.550,9.0


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [ ]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [ ]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [ ]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_SOUTH,Temp_SOUTH,Lag-2,Lag-7
0,16801.433,-11.0,11653.532,12571.309
1,16847.443,-11.4,11515.498,12449.182
2,16997.130,-11.9,11580.872,12295.476
3,17259.598,-12.4,11693.593,12354.318
4,17653.417,-12.9,12210.394,12542.606
...,...,...,...,...
43651,11682.100,9.9,10942.579,11705.914
43652,11442.870,10.2,10694.315,11709.951
43653,11166.770,9.6,10272.441,11663.864
43654,10838.550,9.0,9718.976,11519.032


In [ ]:
# Setting the train-test split ratio
ratio = 0.1

In [ ]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [ ]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [ ]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [ ]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [ ]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [ ]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [ ]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [ ]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [ ]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [ ]:
epochs = 100
batchSize = 32

In [ ]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [ ]:
# Storing the start time of model fitting process
startTime = time.time()

In [ ]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 21s 15ms/step - loss: 0.0125 - mape: 6521.4380 - mae: 0.0869 - val_loss: 0.0082 - val_mape: 49.6771 - val_mae: 0.0710
Epoch 2/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0078 - mape: 5827.5342 - mae: 0.0694 - val_loss: 0.0126 - val_mape: 55.5675 - val_mae: 0.0867
Epoch 3/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0061 - mape: 5097.6714 - mae: 0.0609 - val_loss: 0.0237 - val_mape: 71.5688 - val_mae: 0.1222
Epoch 4/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0054 - mape: 5016.6025 - mae: 0.0571 - val_loss: 0.0138 - val_mape: 60.4707 - val_mae: 0.0925
Epoch 5/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0040 - mape: 4825.3315 - mae: 0.0495 - val_loss: 0.0171 - val_mape: 40.6496 - val_mae: 0.0937
Epoch 6/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0038 - mape: 4396.7188 - mae: 0.0479 - val_l

1105/1105 [==============================] - 17s 15ms/step - loss: 0.0019 - mape: 2394.9402 - mae: 0.0334 - val_loss: 0.0159 - val_mape: 45.9054 - val_mae: 0.0947
Epoch 48/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0019 - mape: 2732.2759 - mae: 0.0330 - val_loss: 0.0145 - val_mape: 42.5478 - val_mae: 0.0900
Epoch 49/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0018 - mape: 2295.1658 - mae: 0.0328 - val_loss: 0.0164 - val_mape: 43.1851 - val_mae: 0.0943
Epoch 50/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0018 - mape: 2185.5479 - mae: 0.0326 - val_loss: 0.0149 - val_mape: 40.5695 - val_mae: 0.0921
Epoch 51/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0018 - mape: 2129.7161 - mae: 0.0323 - val_loss: 0.0147 - val_mape: 43.9578 - val_mae: 0.0907
Epoch 52/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0018 - mape: 2159.4749 - mae: 0.0320 - val_loss: 0.

1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 987.4356 - mae: 0.0249 - val_loss: 0.0151 - val_mape: 44.1455 - val_mae: 0.0910
Epoch 94/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0011 - mape: 1139.9839 - mae: 0.0250 - val_loss: 0.0157 - val_mape: 44.8178 - val_mae: 0.0936
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1313.7899 - mae: 0.0248 - val_loss: 0.0154 - val_mape: 46.4547 - val_mae: 0.0924
Epoch 96/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1426.1646 - mae: 0.0247 - val_loss: 0.0152 - val_mape: 43.7781 - val_mae: 0.0927
Epoch 97/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 965.7338 - mae: 0.0245 - val_loss: 0.0141 - val_mape: 43.7026 - val_mae: 0.0881
Epoch 98/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1268.9988 - mae: 0.0246 - val_loss: 0.01

In [ ]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1703.2855007648468


### MAKING PREDICTIONS AND GENERATING RESULTS

In [ ]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 14ms/step


In [ ]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.20200498, 0.1719141 , 0.16244584, ..., 0.39135388, 0.3642551 ,
       0.3354204 ], dtype=float32)

In [ ]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.41793155, 0.37489408, 0.34738048, ..., 0.37585456, 0.3556864 ,
       0.34057804])

In [ ]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [ ]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,11851.538,8337.512695
1,11151.139,7847.809082
2,10703.378,7693.720703
3,10389.773,7729.737305
4,10352.726,7906.477051
...,...,...
4363,11682.100,11857.454102
4364,11442.870,11716.979492
4365,11166.770,11419.007812
4366,10838.550,10977.998047


In [ ]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

10.00156236724718